# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import ipywidgets as widgets


# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

In [22]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
Df = pd.read_csv("../WeatherPy/Output.csv")

In [10]:
Weather_Data = Df.iloc[:,1:10]
Weather_Data

,City,Lat,Lng,MaxTemp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.12,-134.97,70.61,80.0,18.0,22.06,PF,1.595737e+09
1,mayor pablo lagerenza,-19.93,-60.77,52.77,55.0,98.0,1.48,PY,1.595737e+09
2,lagoa,39.05,-27.98,71.37,88.0,0.0,17.67,PT,1.595737e+09
3,komsomolskiy,40.43,71.72,84.20,37.0,19.0,9.17,UZ,1.595737e+09
4,mahebourg,-20.41,57.70,69.80,68.0,75.0,16.11,MU,1.595737e+09
...,...,...,...,...,...,...,...,...,...
535,saint-marc-des-carrieres,46.68,-72.05,75.00,70.0,40.0,2.24,CA,1.595737e+09
536,caconda,-13.73,15.07,43.41,84.0,2.0,1.28,AO,1.595737e+09
537,aleksandrov gay,50.15,48.57,72.03,41.0,0.0,4.63,RU,1.595737e+09
538,wagar,16.15,36.20,80.62,62.0,100.0,17.29,SD,1.595737e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
locations = Weather_Data[["Lat","Lng"]]
Weights = Weather_Data["Humidity"].astype(float)

In [12]:

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
#fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=Weights, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
Hotel_df = Weather_Data.loc[(Weather_Data["MaxTemp"] >70) & (Weather_Data["MaxTemp"]<80) & (Weather_Data["Wind Speed"]<10) & (Weather_Data["Cloudiness"]==0)]

In [14]:
Hotel_df.reset_index(inplace=True)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = Hotel_df.iloc[:,1:10]

In [16]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,MaxTemp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,yalta,44.50,34.17,75.63,52.0,0.0,5.61,UA,1.595481e+09,
1,tukrah,32.53,20.58,76.95,76.0,0.0,9.24,LY,1.595481e+09,
2,volax,41.32,24.00,79.00,64.0,0.0,2.24,GR,1.595481e+09,
3,gat,31.61,34.76,77.00,62.0,0.0,3.78,IL,1.595481e+09,
4,jawa,31.85,35.94,78.80,39.0,0.0,5.59,JO,1.595481e+09,
5,troitskoye,52.98,84.68,70.99,50.0,0.0,4.85,RU,1.595481e+09,
6,faya,18.39,42.45,73.40,49.0,0.0,5.93,SA,1.595481e+09,
7,kerman,30.28,57.08,77.00,13.0,0.0,4.70,IR,1.595481e+09,
8,usinsk,65.99,57.53,75.88,48.0,0.0,8.41,RU,1.595481e+09,
9,tiznit,29.58,-9.50,76.87,37.0,0.0,1.03,MA,1.595481e+09,


In [17]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "key" : g_key,
    "radius": 5000,
    "types":"hotel" ,
    "keyword" :"Nearby Hotel"}
for index,row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    response = requests.get(url,params=params).json()
    if (len(response["results"]) > 0):
        hotel_df.loc[index,"Hotel Name"] = response["results"][0]["name"]
print("DONE")
hotel_df

DONE


,City,Lat,Lng,MaxTemp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,yalta,44.50,34.17,75.63,52.0,0.0,5.61,UA,1.595481e+09,Massandra
1,tukrah,32.53,20.58,76.95,76.0,0.0,9.24,LY,1.595481e+09,
2,volax,41.32,24.00,79.00,64.0,0.0,2.24,GR,1.595481e+09,Monopati E6
3,gat,31.61,34.76,77.00,62.0,0.0,3.78,IL,1.595481e+09,Desert Gat
4,jawa,31.85,35.94,78.80,39.0,0.0,5.59,JO,1.595481e+09,Opal Hotel
5,troitskoye,52.98,84.68,70.99,50.0,0.0,4.85,RU,1.595481e+09,
6,faya,18.39,42.45,73.40,49.0,0.0,5.93,SA,1.595481e+09,
7,kerman,30.28,57.08,77.00,13.0,0.0,4.70,IR,1.595481e+09,
8,usinsk,65.99,57.53,75.88,48.0,0.0,8.41,RU,1.595481e+09,"Гостиница ""Венеция"""
9,tiznit,29.58,-9.50,76.87,37.0,0.0,1.03,MA,1.595481e+09,


In [18]:
hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!="")]

In [19]:
hotel_df

,City,Lat,Lng,MaxTemp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,yalta,44.50,34.17,75.63,52.0,0.0,5.61,UA,1.595481e+09,Massandra
2,volax,41.32,24.00,79.00,64.0,0.0,2.24,GR,1.595481e+09,Monopati E6
3,gat,31.61,34.76,77.00,62.0,0.0,3.78,IL,1.595481e+09,Desert Gat
4,jawa,31.85,35.94,78.80,39.0,0.0,5.59,JO,1.595481e+09,Opal Hotel
8,usinsk,65.99,57.53,75.88,48.0,0.0,8.41,RU,1.595481e+09,"Гостиница ""Венеция"""
10,fiumicino,41.77,12.23,75.00,72.0,0.0,2.24,IT,1.595481e+09,Hilton Garden Inn Rome Airport


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…